### summarize compositional lasso results on Independent Simulation Scenarios for binary outcome

In [1]:
dir = '/panfs/panfs1.ucsd.edu/panscratch/lij014/Stability_2020/sim_data'

In [2]:
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [3]:
length(files)

[1] 16

In [4]:
avg_FDR = NULL
table_toe = NULL
tmp_num_select = rep(0, length(files))
for (i in 1:length(files)){
    print(paste0('indx: ', i))
    load(paste0(dir, '/binary_update/ind_GenCompLasso_binary_', i, '.RData')) 
    
    table_toe = rbind(table_toe, results_ind_GenCompLasso[c('n', 'p', 'rou', 'FP', 'FN', 'ROC', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_ind_GenCompLasso$Stab.table))
    
    # calculate FDR
    load(file_name, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_GenCompLasso$Stab.table
    FDR = NULL # false positive rate
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}
table_toe = as.data.frame(table_toe)
table_toe$num_select = tmp_num_select
table_toe$FDR = round(avg_FDR,2)

[1] "indx: 1"
[1] "indx: 2"
[1] "indx: 3"
[1] "indx: 4"
[1] "indx: 5"
[1] "indx: 6"
[1] "indx: 7"
[1] "indx: 8"
[1] "indx: 9"
[1] "indx: 10"
[1] "indx: 11"
[1] "indx: 12"
[1] "indx: 13"
[1] "indx: 14"
[1] "indx: 15"
[1] "indx: 16"


In [5]:
head(table_toe)

,n,p,rou,FP,FN,ROC,Stab,num_select,FDR
,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>,<dbl>
1,50,50,0,16.38 ( 0.96 ),0.61 ( 0.06 ),0.98 ( 0 ),0.13,21.77,0.71
2,100,50,0,7.94 ( 0.54 ),0.34 ( 0.06 ),0.93 ( 0 ),0.32,13.60,0.54
3,500,50,0,2.67 ( 0.6 ),0.1 ( 0.03 ),0.91 ( 0 ),0.63,8.57,0.19
4,1000,50,0,1.61 ( 0.48 ),0.04 ( 0.02 ),0.91 ( 0 ),0.75,7.57,0.09
5,50,100,0,39.18 ( 2.14 ),0.63 ( 0.07 ),0.99 ( 0 ),0.06,44.55,0.85
6,100,100,0,16.29 ( 1.16 ),0.47 ( 0.06 ),0.94 ( 0 ),0.2,21.82,0.70


In [6]:
tail(table_toe)

,n,p,rou,FP,FN,ROC,Stab,num_select,FDR
,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>,<dbl>
11,500,500,0,8.32 ( 1.15 ),0.1 ( 0.03 ),0.91 ( 0 ),0.43,14.22,0.41
12,1000,500,0,4.63 ( 1.57 ),0 ( 0 ),0.91 ( 0 ),0.6,10.63,0.21
13,50,1000,0,406.32 ( 7.33 ),0.73 ( 0.07 ),1 ( 0 ),0.01,411.59,0.99
14,100,1000,0,283.24 ( 7.31 ),0.49 ( 0.06 ),1 ( 0 ),0.02,288.75,0.98
15,500,1000,0,9.4 ( 1.07 ),0.07 ( 0.03 ),0.91 ( 0 ),0.4,15.33,0.47
16,1000,1000,0,1.91 ( 0.29 ),0.01 ( 0.01 ),0.91 ( 0 ),0.82,7.90,0.18


In [7]:
# export result
result.table_toe <- apply(table_toe,2,as.character)
rownames(result.table_toe) = rownames(table_toe)
result.table_toe = as.data.frame(result.table_toe)

# extract numbers only for 'n' & 'p'
result.table_toe$n = tidyr::extract_numeric(result.table_toe$n)
result.table_toe$p = tidyr::extract_numeric(result.table_toe$p)
result.table_toe$ratio = result.table_toe$p / result.table_toe$n

result.table_toe = result.table_toe[c('n', 'p', 'rou', 'ratio', 'Stab', 'ROC', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_toe)[1:4] = c('N', 'P', 'Corr', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead

extract_numeric() is deprecated: please use readr::parse_number() instead



In [8]:
# convert interested measurements to be numeric
result.table_toe$Stab = as.numeric(as.character(result.table_toe$Stab))
# result.table_toe$ROC_mean = as.numeric(substr(result.table_toe$ROC, start=1, stop=4))
# result.table_toe$FP_mean = as.numeric(substr(result.table_toe$FP, start=1, stop=4))
# result.table_toe$FN_mean = as.numeric(substr(result.table_toe$FN, start=1, stop=4))
# result.table_toe$FN_mean[is.na(result.table_toe$FN_mean)] = 0
result.table_toe$num_select = as.numeric(as.character(result.table_toe$num_select))

result.table_toe$ROC_mean = as.numeric(sub("\\(.*", "", result.table_toe$ROC))
result.table_toe$FP_mean = as.numeric(sub("\\(.*", "", result.table_toe$FP))
result.table_toe$FN_mean = as.numeric(sub("\\(.*", "", result.table_toe$FN))

In [9]:
# check whether missing values exists
result.table_toe[rowSums(is.na(result.table_toe)) > 0,]

N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>


In [10]:
head(result.table_toe)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,50,50,0,1.00,0.13,0.98 ( 0 ),16.38 ( 0.96 ),0.61 ( 0.06 ),21.77,0.71,0.98,16.38,0.61
2,100,50,0,0.50,0.32,0.93 ( 0 ),7.94 ( 0.54 ),0.34 ( 0.06 ),13.60,0.54,0.93,7.94,0.34
3,500,50,0,0.10,0.63,0.91 ( 0 ),2.67 ( 0.6 ),0.1 ( 0.03 ),8.57,0.19,0.91,2.67,0.10
4,1000,50,0,0.05,0.75,0.91 ( 0 ),1.61 ( 0.48 ),0.04 ( 0.02 ),7.57,0.09,0.91,1.61,0.04
5,50,100,0,2.00,0.06,0.99 ( 0 ),39.18 ( 2.14 ),0.63 ( 0.07 ),44.55,0.85,0.99,39.18,0.63
6,100,100,0,1.00,0.20,0.94 ( 0 ),16.29 ( 1.16 ),0.47 ( 0.06 ),21.82,0.7,0.94,16.29,0.47


In [11]:
tail(result.table_toe)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
11,500,500,0,1.0,0.43,0.91 ( 0 ),8.32 ( 1.15 ),0.1 ( 0.03 ),14.22,0.41,0.91,8.32,0.10
12,1000,500,0,0.5,0.60,0.91 ( 0 ),4.63 ( 1.57 ),0 ( 0 ),10.63,0.21,0.91,4.63,0.00
13,50,1000,0,20.0,0.01,1 ( 0 ),406.32 ( 7.33 ),0.73 ( 0.07 ),411.59,0.99,1.00,406.32,0.73
14,100,1000,0,10.0,0.02,1 ( 0 ),283.24 ( 7.31 ),0.49 ( 0.06 ),288.75,0.98,1.00,283.24,0.49
15,500,1000,0,2.0,0.40,0.91 ( 0 ),9.4 ( 1.07 ),0.07 ( 0.03 ),15.33,0.47,0.91,9.40,0.07
16,1000,1000,0,1.0,0.82,0.91 ( 0 ),1.91 ( 0.29 ),0.01 ( 0.01 ),7.90,0.18,0.91,1.91,0.01


In [12]:
result.table_toe

## export
write.table(result.table_toe, '../results_summary_updated/sim_ind_GencompLasso_binary.txt', sep='\t', row.names=F)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,50,50,0,1.00,0.13,0.98 ( 0 ),16.38 ( 0.96 ),0.61 ( 0.06 ),21.77,0.71,0.98,16.38,0.61
2,100,50,0,0.50,0.32,0.93 ( 0 ),7.94 ( 0.54 ),0.34 ( 0.06 ),13.60,0.54,0.93,7.94,0.34
3,500,50,0,0.10,0.63,0.91 ( 0 ),2.67 ( 0.6 ),0.1 ( 0.03 ),8.57,0.19,0.91,2.67,0.10
4,1000,50,0,0.05,0.75,0.91 ( 0 ),1.61 ( 0.48 ),0.04 ( 0.02 ),7.57,0.09,0.91,1.61,0.04
5,50,100,0,2.00,0.06,0.99 ( 0 ),39.18 ( 2.14 ),0.63 ( 0.07 ),44.55,0.85,0.99,39.18,0.63
6,100,100,0,1.00,0.20,0.94 ( 0 ),16.29 ( 1.16 ),0.47 ( 0.06 ),21.82,0.7,0.94,16.29,0.47
7,500,100,0,0.20,0.47,0.91 ( 0 ),5.63 ( 0.84 ),0.13 ( 0.03 ),11.50,0.3,0.91,5.63,0.13
8,1000,100,0,0.10,0.43,0.91 ( 0 ),6.87 ( 1.8 ),0 ( 0 ),12.87,0.17,0.91,6.87,0.00
9,50,500,0,10.00,0.01,1 ( 0 ),251.02 ( 6.8 ),0.36 ( 0.06 ),256.66,0.98,1.00,251.02,0.36
